In [3]:
import random
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime

# Configure Selenium WebDriver
CHROMEDRIVER_PATH = "/Users/mmmymelody/Desktop/chromedriver-mac-arm64/chromedriver"
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])

# Spoof User-Agent
USER_AGENTS = [
    # Chrome on Windows
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.60 Safari/537.36",

    # Firefox on Windows
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0",

    # Safari on Mac
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15",

    # Chrome on Mac
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.60 Safari/537.36",

    # Mobile Safari on iPhone
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",

    # Chrome on Android
    "Mozilla/5.0 (Linux; Android 13; Pixel 6a) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.60 Mobile Safari/537.36",

    # Samsung Internet
    "Mozilla/5.0 (Linux; Android 12; SAMSUNG SM-G996U1) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/24.0 Chrome/110.0.5481.77 Mobile Safari/537.36",

    # Edge on Windows
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.60 Safari/537.36 Edg/124.0.2478.67"
]


options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")
options.add_argument("--start-maximized")
options.add_argument("--window-size=1920,1080")

# Start WebDriver and Visit Amazon
service = Service(CHROMEDRIVER_PATH)
print("Start WebDriver...")
driver = webdriver.Chrome(service=service, options=options)
url = "https://www.amazon.com/s?k=sunscreen+spf"
driver.get(url)
time.sleep(random.uniform(3, 5))

def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(7):
        driver.execute_script("window.scrollBy(0, 800);")
        time.sleep(random.uniform(2, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def go_to_next_page(driver):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 's-pagination-next')]"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("Successfully clicked 'Next' button")
        time.sleep(random.uniform(3, 5))
        return True
    except Exception as e:
        print("No 'Next' button found", e)
        return False

def extract_details_from_product_page(link):
    try:
        driver.get(link)
        time.sleep(random.uniform(2, 4))
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        brand_span = soup.select_one("tr.po-brand span.po-break-word")
        brand = brand_span.get_text(strip=True) if brand_span else "N/A"

        discount_elem = soup.select_one("span.savingsPercentage")
        is_on_sale = 1 if discount_elem else 0

        review_count_span = soup.find('span', id='acrCustomerReviewText')
        if review_count_span:
            review_text = review_count_span.get_text(strip=True)
            match = re.search(r'\d[\d,]*', review_text)
            review_counts = match.group(0).replace(",", "") if match else "N/A"
        else:
            review_counts = "N/A"

        return brand, is_on_sale, review_counts
    except Exception as e:
        print(f"Error getting details from {link}: {e}")
        return "N/A", 0, "N/A"
    finally:
        driver.back()

def scrape_page():
    soup = BeautifulSoup(driver.page_source, "html.parser")
    product_list = []
    product_divs = soup.find_all('div', class_=lambda x: x and 's-card-container' in x)

    for product_div in product_divs:
        try:
            name_elem = product_div.find('a', {'class': 'a-link-normal s-line-clamp-3 s-link-style a-text-normal'})
            product_name = name_elem.get_text(strip=True) if name_elem else "N/A"
            product_link = "https://www.amazon.com" + name_elem['href'] if name_elem else None

            price_whole = product_div.find('span', {'class': 'a-price-whole'})
            price_frac = product_div.find('span', {'class': 'a-price-fraction'})
            current_price = f"{price_whole.get_text(strip=True)}.{price_frac.get_text(strip=True)}" if price_whole and price_frac else "N/A"

            star_elem = product_div.find('a', {'aria-label': True})
            if star_elem:
                rating_text = star_elem['aria-label']
                match = re.search(r'([\d.]+) out of', rating_text)
                star_rating = match.group(1) if match else "N/A"
            else:
                star_rating = "N/A"

            unit_price = "N/A"
            unit_raw_text = ""
            secondary_spans = product_div.find_all('span', class_='a-size-base a-color-secondary')
            for span in secondary_spans:
                if span.find('span', class_='a-price a-text-price'):
                    try:
                        price_val = span.find('span', class_='a-offscreen').get_text(strip=True)
                        unit_raw_text = span.get_text()
                        unit_price = re.search(r"[\d.]+", price_val).group(0) if price_val else "N/A"
                        break
                    except:
                        continue

            # skip if unit_price is N/A or unit is count
            if unit_price == "N/A" or "count" in unit_raw_text.lower():
                print(f"Skipping '{product_name}' due to invalid unit price")
                continue

            # check delivery time
            is_3day_delivery = 0
            try:
                delivery_span = product_div.find('span', attrs={"aria-label": re.compile(r'Or fastest delivery')})
                if delivery_span:
                    aria_text = delivery_span['aria-label']
                    today = datetime.now()
                    if "Today" in aria_text or "Tomorrow" in aria_text:
                        is_3day_delivery = 1
                    else:
                        date_match = re.search(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}', aria_text)
                        if date_match:
                            delivery_date_str = f"{date_match.group(0)} {today.year}"
                            delivery_date = datetime.strptime(delivery_date_str, "%b %d %Y")
                            is_3day_delivery = 1 if (delivery_date - today).days <= 3 else 0
            except:
                is_3day_delivery = 0

            brand, is_on_sale, review_counts = extract_details_from_product_page(product_link) if product_link else ("N/A", 0, "N/A")
            if review_counts == "N/A":
                print(f"Skipping '{product_name}' due to missing review count")
                continue

            product_list.append({
                "product_category": "sunscreen",
                "product_name": product_name,
                "brand": brand,
                "current_price": current_price,
                "unit_price": unit_price,
                "star_rating": star_rating,
                "review_counts": review_counts,
                "is_on_sale": is_on_sale,
                "is_free_shipping": 1,
                "is_3day_delivery": is_3day_delivery,
                "product_link": product_link,
                "platform": "Amazon"
            })

        except Exception as e:
            print(f"Skipping product: {e}")
            continue

    print(f"Collected {len(product_list)} products from current page")
    return product_list

# Run scraping loop
all_products = []
max_pages = 5
current_page = 1

while current_page <= max_pages:
    print(f"Fetching page {current_page}...")
    scroll_and_wait()
    scraped = scrape_page()
    all_products.extend(scraped)
    print(f"Page {current_page} done, total collected so far: {len(all_products)}")
    if not go_to_next_page(driver):
        break
    current_page += 1

print(f"Final total to save: {len(all_products)}")
# driver.quit()

# Output columns
column_order = [
    "product_name", "brand", "current_price", "unit_price", "star_rating",
    "review_counts", "is_on_sale", "is_free_shipping", "is_3day_delivery",
    "product_category", "product_link", "platform"
]

df = pd.DataFrame(all_products)[column_order]
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
print(df)

df.to_csv("Amazon_sunscreen_data.csv", index=False)


Start WebDriver...
Fetching page 1...
Error getting details from https://www.amazon.comhttps://aax-us-iad.amazon.com/x/c/JPdkJ5Ed4iYGFCewC7ovFpYAAAGWNbdKZwEAAAH2AQBvbm9fdHhuX2JpZDQgICBvbm9fdHhuX2ltcDEgICCWlaNr/clv1_CEuOPUxokZA0iHrVC70UxSHqG3FZXYsIlsgu0SGLp9xxx39V6EKddvw18nHz9hvWjq0yzpNb3XTPsunPNGcAVhJaMSKrTcvRHHAwRB5Yb0agTLl4skfB8IkGXHyWUUJ15WjgQNVxWl3UxW5Xadw56em4GwK6_h7n_zqlNqFHKBCQi6XSzA1lHALidMO1d6EiWTUdd8DRkpZWRgEhKdZHfvAWlMWy1t4nBFzaMqosjFVarp9vkfW-XQ8ljlikgQEOSFTEBcfDpgdjzgs8FvtMKCmPU3Cpj0r2Nf2y9rHYd7UDeHi-za9el2hZFG6zfKGs6uM31cxnToHfm1m3ZJnd70TZ-hbNQTlPUUgPPX1HiA/https://www.amazon.com/Neutrogena-Sunscreen-Lightweight-Non-Greasy-Oxybenzone-Free/dp/B0DG2QN9XF/ref=sxin_17_sbv_search_btf?content-id=amzn1.sym.7032aefd-3c59-4a1e-aaf4-8d3a944207a4%3Aamzn1.sym.7032aefd-3c59-4a1e-aaf4-8d3a944207a4&cv_ct_cx=sunscreen+spf&keywords=sunscreen+spf&pd_rd_i=B0DG2QN9XF&pd_rd_r=11a40f9a-19b3-40a3-8daf-928b8c010229&pd_rd_w=QWhs8&pd_rd_wg=7nwGX&pf_rd_p=7032aefd-3c59-4a1e-aaf4-8d3a944207a4&pf_rd_r

In [6]:
import pandas as pd
import re
import glob
import os
import sys
from typing import Optional, List


def _normalise_price_string(val: str) -> Optional[float]:
    """Return float rounded to 2 dp, or None if not numeric."""
    if pd.isna(val):
        return None
    s = re.sub(r"[^0-9.]", "", str(val))          # keep digits/dots
    if not s:
        return None
    s = re.sub(r"\.+", ".", s)                    # collapse multiple dots
    if s.count(".") > 1:
        head, *tail = s.split(".")
        s = head + "." + "".join(tail)
    try:
        return round(float(s), 2)
    except ValueError:
        return None


def clean_file(path: str) -> None:
    df = pd.read_csv(path)

    # ----- price columns ---------------------------------------------------
    for col in ("current_price", "unit_price"):
        if col in df.columns:
            df[col] = df[col].apply(_normalise_price_string)

    # ----- duplicates ------------------------------------------------------
    before = len(df)
    df = df.drop_duplicates()
    removed = before - len(df)
    if removed:
        print(f"   → Removed {removed} duplicate rows")

    # ----- output CSV ------------------------------------------------------
    base = os.path.splitext(path)[0] + "_clean"
    csv_path = base + ".csv"
    df.to_csv(csv_path, index=False, float_format="%.2f")
    print(f"✔  CSV  saved → {csv_path}")

    # ----- output Excel with formatting -----------------------------------
    xlsx_path = base + ".xlsx"
    with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as writer:
        df.to_excel(writer, index=False, sheet_name="Sheet1")
        workbook  = writer.book
        worksheet = writer.sheets["Sheet1"]
        money_fmt = workbook.add_format({"num_format": "0.00"})
        header_fmt = workbook.add_format({"bold": True})

        # Apply formats
        for idx, col in enumerate(df.columns):
            width = max(df[col].astype(str).map(len).max(), len(col)) + 2
            if col in ("current_price", "unit_price"):
                worksheet.set_column(idx, idx, width, money_fmt)
            else:
                worksheet.set_column(idx, idx, width)
            worksheet.write(0, idx, col, header_fmt)
    print(f"✔  Excel saved → {xlsx_path}\n")


def _extract_folder_arg(args: List[str]) -> str:
    for arg in args:
        if not arg.startswith("-"):
            return arg
    return os.path.join(os.getcwd(), "Data")

def main() -> None:
    folder = _extract_folder_arg(sys.argv[1:])
    print(f"🔍 Searching in: {folder}")

    pattern = os.path.join(folder, "**", "Amazon_*_data.csv")
    files = glob.glob(pattern, recursive=True)

    if not files:
        print(f"No CSV files found matching {pattern}")
        return

    for file_path in files:
        print(f"→ Processing {os.path.relpath(file_path, folder)}")
        try:
            clean_file(file_path)
        except Exception as exc:
            print(f"⚠️  Skipped {file_path}: {exc}")

if __name__ == "__main__":
    main()



🔍 Searching in: /Users/mmmymelody/Desktop/e-Recommend/Data
→ Processing Amazon_eyecream_data.csv
✔  CSV  saved → /Users/mmmymelody/Desktop/e-Recommend/Data/Amazon_eyecream_data_clean.csv
⚠️  Skipped /Users/mmmymelody/Desktop/e-Recommend/Data/Amazon_eyecream_data.csv: No module named 'xlsxwriter'
→ Processing Amazon_handcream_data.csv
✔  CSV  saved → /Users/mmmymelody/Desktop/e-Recommend/Data/Amazon_handcream_data_clean.csv
⚠️  Skipped /Users/mmmymelody/Desktop/e-Recommend/Data/Amazon_handcream_data.csv: No module named 'xlsxwriter'
→ Processing Amazon_moisturizer_data.csv
   → Removed 7 duplicate rows
✔  CSV  saved → /Users/mmmymelody/Desktop/e-Recommend/Data/Amazon_moisturizer_data_clean.csv
⚠️  Skipped /Users/mmmymelody/Desktop/e-Recommend/Data/Amazon_moisturizer_data.csv: No module named 'xlsxwriter'
→ Processing Amazon_facialmask_data.csv
✔  CSV  saved → /Users/mmmymelody/Desktop/e-Recommend/Data/Amazon_facialmask_data_clean.csv
⚠️  Skipped /Users/mmmymelody/Desktop/e-Recommend/Dat

In [1]:
!pip install pandas

In [ ]:
import pandas as pd

df1 = pd.DataFrame(all_products)

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False) 
print(df1)

In [ ]:
import pandas as pd

df2 = pd.DataFrame(all_products2)

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False) 
print(df2)

df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
pd.set_option("display.max_rows", None)  # Show all rows
df_original = df.copy()

In [46]:
df.head(5)

,product_name,brand,price,unit_price,star_rating,Delivery time (3days or not),free_shipping,num_ratings,discount,product_url,platform,keyword
0,"Amazon Basics Hydrating Facial Cleanser, 12 Fl...",Amazon Basics,$8.13,$0.68/Fl,4.2/5,1,1,457,-5%,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Amazon,cleanser
1,La Roche-Posay Toleriane Purifying Foaming Fac...,La Roche-Posay,$15.99,$1.18/Fl,4.7/5,1,1,39497,-11%,https://www.amazon.com/Roche-Posay-Toleriane-P...,Amazon,cleanser
2,La Roche-Posay Toleriane Hydrating Gentle Face...,La Roche-Posay,$15.99,$1.18/Fl,4.6/5,1,1,37159,-11%,https://www.amazon.com/Roche-Posay-Toleriane-H...,Amazon,cleanser
3,"CeraVe Hydrating Facial Cleanser, Moisturizing...",CeraVe,$13.93,$0.87/Fl,4.7/5,1,1,120412,-23%,https://www.amazon.com/CeraVe-Hydrating-Facial...,Amazon,cleanser
4,"IMAGE Skincare, Ormedic pH Balancing Facial Cl...",IMAGE Skincare,$39.00,$6.50/Ounce,4.7/5,0,1,586,-5%,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Amazon,cleanser


In [48]:
df.tail(5)

,product_name,brand,price,unit_price,star_rating,Delivery time (3days or not),free_shipping,num_ratings,discount,product_url,platform,keyword
275,N/A,N/A,$17.77,$0.36/Count,4.5/5,0,1,N/A,N/A,None,Amazon,face mask
276,N/A,N/A,$17.99,$0.30/Count,4.6/5,0,1,N/A,N/A,None,Amazon,face mask
277,N/A,N/A,$21.07,$0.21/Count,4.4/5,0,1,N/A,N/A,None,Amazon,face mask
278,N/A,N/A,$17.99,$0.90/Count,4.5/5,0,1,N/A,N/A,None,Amazon,face mask
279,N/A,N/A,$12.99,N/A,4.4/5,0,1,N/A,N/A,None,Amazon,face mask


In [35]:
df.sample(5)

,product_name,brand,price,unit_price,star_rating,Delivery time (3days or not),free_shipping,num_ratings,discount,product_url,platform
44,Clean & Clear 2-Pack Oil-Free Facial Cleansers...,Clean & Clear,$9.66,$1.21/Fl,4.6/5,1,1,38927,-34%,https://www.amazon.com/2-Pack-Oil-Free-Cleanse...,Amazon
29,"Dr. Hauschka Soothing Cleansing Milk, 1 Fl Oz",Dr. Hauschka,$13.00,$13.00/Fl,4.4/5,1,1,41,N/A,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Amazon
6,La Roche-Posay Effaclar Purifying Foaming Gel ...,La Roche-Posay,$22.99,$1.70/Fl,4.6/5,0,1,44108,N/A,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Amazon
14,"Cetaphil Face Wash, Daily Facial Cleanser for ...",Idel,$13.44,$0.67/Fl,4.7/5,1,1,18602,-10%,https://www.amazon.com/CETAPHIL-Cleanser-Sensi...,Amazon
55,"Youth To The People Superfood Facial Cleanser,...",Youth To The People,$16.00,$8.00/Fl,4.6/5,1,1,8363,-5%,https://www.amazon.com/Youth-People-Superfood-...,Amazon


In [16]:
!pip install -qqq mysqlclient
!pip install -qqq -U sql_magic

In [17]:
pip install pymysql


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
!pip install sqlalchemy
!{sys.executable} -m pip install sqlalchemy


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [19]:
from sqlalchemy import create_engine

In [ ]:
conn_string = 'mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user="dbmh",
    password="jelIyvfzqwNBWJXpC9bJTQ==",
    host = 'jsedocc7.scrc.nyu.edu',
    port = 3306,
    encoding = 'utf-8',
    db = 'dbmh'
)
engine = create_engine(conn_string)
pd.read_sql('SHOW TABLES', con=engine)
df.to_sql('Products_Table', con=engine, if_exists='append', index=False)

In [ ]:
import os, glob, sys
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

csv_dir = "/Users/mmmymelody/Desktop/e-Recommend/Clean_data"

user     = "dbmh"
password = os.getenv("DB_PWD")     
host     = "jsedoc7.scrc.nyu.edu"    
port     = 3306
db       = "dbmh"
table    = "Products_Table"

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4",
    pool_pre_ping=True,
    connect_args={"local_infile": 1, "connect_timeout": 10}
)

csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))
if not csv_files:
    sys.exit(f"No csv file found in {csv_dir}")

for path in csv_files:
    try:
        print(f"processing {os.path.basename(path)} ...")
        df = (
            pd.read_csv(path)
              .rename(columns=str.lower) 
              .drop_duplicates()           
        )


        df.to_sql(
            table,
            con=engine,
            if_exists="append",          
            index=False,
            chunksize=1000,
            method="multi"
        )
        print(f"finished line {len(df)} ")
    except (pd.errors.ParserError, SQLAlchemyError) as e:
        print(f"skip {path}：{e}")

print("All finished")


In [ ]:
pd.read_sql('SELECT * FROM Products_Table',con= engine)